Depending on your choice of python notebook, you are going to need to install and load the following packages to perform topic modeling. Among the SoMe, data science team we use a wide range of notebook options, from Azure to Jupyter labs and notebook. My personal favorite however is Google Colab. I recommend running all the following snippets on either Google Colab or Jupyter notebook. I’m also going to link the Github repository for this project and a link to the final notebook we used for your reference.


In [1]:
# Installations
import sys
if 'google.colab' in sys.modules:
    !pip install emoji --upgrade
    !pip install pandas-profiling==2.*
    !pip install plotly==4.*
    !python -m spacy download en_core_web_lg
    !pip install pyldavis
    !pip install gensim
    !pip install chart_studio
    !pip install --upgrade autopep8

Requirement already up-to-date: emoji in /usr/local/lib/python3.7/dist-packages (1.2.0)
  Using cached https://files.pythonhosted.org/packages/2d/e6/c3ad340cbbf8b93f84922c80f92359a6e6c7f88919c4d4353b8b7fe17f8d/pandas_profiling-2.13.0-py2.py3-none-any.whl
  Found existing installation: pandas-profiling 3.0.0
    Uninstalling pandas-profiling-3.0.0:
      Successfully uninstalled pandas-profiling-3.0.0
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')
Requirement already up-to-date: autopep8 in /usr/local/lib/python3.7/dist-packages (1.5.7)


In [2]:
# Required Libraries

#Base and Cleaning 
import json
import requests
import pandas as pd
import numpy as np
import emoji
import regex
import re
import string
from collections import Counter

#Visualizations
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
import pyLDAvis
import pyLDAvis.gensim_models
import pyLDAvis.gensim_models as gensim_models
#import pyLDAvis.gensim
import chart_studio
import chart_studio.plotly as py 
import chart_studio.tools as tls

#Natural Language Processing (NLP)
import spacy
import gensim
from spacy.tokenizer import Tokenizer
from gensim.corpora import Dictionary
from gensim.models.ldamulticore import LdaMulticore
from gensim.models.coherencemodel import CoherenceModel
from gensim.parsing.preprocessing import STOPWORDS as SW
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint
from wordcloud import STOPWORDS
stopwords = set(STOPWORDS)

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning:

Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working

/usr/local/lib/python3.7/dist-packages/past/builtins/misc.py:4: DeprecationWarning:

Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_least_angle.py:30: DeprecationWarning:

`np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_least_angle.py:169: Depr

In [3]:
#!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


link = 'https://drive.google.com/file/d/1cvDvw4pZMQScJdevP7vg7129k6725gM5/view?usp=sharing' #the shareable link

id = '1cvDvw4pZMQScJdevP7vg7129k6725gM5'

print (id) # Verify that you have everything after '='

downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('Covid_vaccine_Tagged.csv')  
df3 = pd.read_csv('Covid_vaccine_Tagged.csv')

df3 = df3[df3.Analysis.notna()]

df3 = df3.reset_index(drop = True)

1cvDvw4pZMQScJdevP7vg7129k6725gM5


# Data Cleaning

After pulling and refining the data from the Twitter api and importing the required packages, we had to clean the data from emojis and urls so we can tokenize it for the next steps. You can find the cleaning syntaxes we used in the below snippet:

In [4]:
def give_emoji_free_text(text):
    """
    Removes emoji's from tweets
    Accepts:
        Text (tweets)
    Returns:
        Text (emoji free tweets)
    """
    emoji_list = [c for c in text if c in emoji.UNICODE_EMOJI]
    clean_text = ' '.join([str for str in text.split() if not any(i in str for i in emoji_list)])
    return clean_text

def url_free_text(text):
    '''
    Cleans text from urls
    '''
    text = re.sub(r'http\S+', '', text)
    return text

# Apply the function above and get tweets free of emoji's
call_emoji_free = lambda x: give_emoji_free_text(x)

# Apply `call_emoji_free` which calls the function to remove all emoji's
df3['emoji_free_tweets'] = df3['text'].apply(call_emoji_free)

#Create a new column with url free tweets
df3['url_free_tweets'] = df3['emoji_free_tweets'].apply(url_free_text)

In [22]:
df3

,id,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,retweets,favorites,is_retweet,processed_tweets,Polarity,Analysis,emoji_free_tweets,url_free_tweets,tokens,tokens_back_to_text,lemmas,lemmas_back_to_text,lemma_tokens
0,1340539111971516416,Rachel Roh,"La Crescenta-Montrose, CA",Aggregator of Asian American news; scanning di...,2009-04-08 17:52:46,405,1692,3247,False,2020-12-20 06:06:44,Same folks said daikon paste could treat a cyt...,['PfizerBioNTech'],Twitter for Android,0,0.0,False,folk said daikon paste could treat cytokine st...,0.292167,Negative,Same folks said daikon paste could treat a cyt...,Same folks said daikon paste could treat a cyt...,"[folks, said, daikon, paste, treat, cytokine, ...",folks said daikon paste treat cytokine storm #...,"[folk, say, daikon, paste, treat, cytokine, st...",folk say daikon paste treat cytokine storm pfi...,"[folk, say, daikon, paste, treat, cytokine, st..."
1,1338158543359250433,Albert Fong,"San Francisco, CA","Marketing dude, tech geek, heavy metal & '80s ...",2009-09-21 15:27:30,834,666,178,False,2020-12-13 16:27:13,While the world has been on the wrong side of ...,NaN,Twitter Web App,1,1.0,False,world wrong side history year hopefully bigges...,0.753310,Positive,While the world has been on the wrong side of ...,While the world has been on the wrong side of ...,"[world, wrong, history, year,, hopefully,, big...","world wrong history year, hopefully, biggest v...","[world, wrong, history, year, hopefully, big, ...",world wrong history year hopefully big vaccina...,"[world, wrong, history, year, hopefully, big, ..."
2,1337858199140118533,eli🇱🇹🇪🇺👌,Your Bed,"heil, hydra 🖐☺",2020-06-25 23:30:28,10,88,155,False,2020-12-12 20:33:45,#coronavirus #SputnikV #AstraZeneca #PfizerBio...,"['coronavirus', 'SputnikV', 'AstraZeneca', 'Pf...",Twitter for Android,0,0.0,False,coronavirus sputnikv astrazeneca pfizerbiontec...,0.364838,Negative,#coronavirus #SputnikV #AstraZeneca #PfizerBio...,#coronavirus #SputnikV #AstraZeneca #PfizerBio...,"[#coronavirus, #sputnikv, #astrazeneca, #pfize...",#coronavirus #sputnikv #astrazeneca #pfizerbio...,"[coronavirus, sputnikv, astrazeneca, pfizerbio...",coronavirus sputnikv astrazeneca pfizerbiontec...,"[coronavirus, sputnikv, astrazeneca, pfizerbio..."
3,1337855739918835717,Charles Adler,"Vancouver, BC - Canada","Hosting ""CharlesAdlerTonight"" Global News Radi...",2008-09-10 11:28:53,49165,3933,21853,True,2020-12-12 20:23:59,"Facts are immutable, Senator, even when you're...",NaN,Twitter Web App,446,2129.0,False,fact immutable senator even youre not ethicall...,0.187278,Negative,"Facts are immutable, Senator, even when you're...","Facts are immutable, Senator, even when you're...","[facts, immutable,, senator,, you're, ethicall...","facts immutable, senator, you're ethically stu...","[fact, immutable, senator, ethically, sturdy, ...",fact immutable senator ethically sturdy acknow...,"[fact, immutable, senator, ethically, sturdy, ..."
4,1337854064604966912,Citizen News Channel,NaN,Citizen News Channel bringing you an alternati...,2020-04-23 17:58:42,152,580,1473,False,2020-12-12 20:17:19,Explain to me again why we need a vaccine @Bor...,"['whereareallthesickpeople', 'PfizerBioNTech']",Twitter for iPhone,0,0.0,False,explain need vaccine whereareallthesickpeople ...,0.782983,Positive,Explain to me again why we need a vaccine @Bor...,Explain to me again why we need a vaccine @Bor...,"[explain, need, vaccine, @borisjohnson, @matth...",explain need vaccine @borisjohnson @matthancoc...,"[explain, need, vaccine, @borisjohnson, @matth...",explain need vaccine @borisjohnson @matthancoc...,"[explain, need, vaccine, borisjohnson, matthan..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69709,1382248484259123205,Russian Mission in Geneva,Geneve,Постпредство России при Отделении ООН и др. ме...,2011-07-26 08:40:47,9862,589,3

As you can see in the table below, there are a lot of emojis and urls you want to remove before tokenizing. Data cleaning is essential for ultimately delivering accurate results, we do not want words from websites or emojis end up in our topic modeling results since they deliver almost no value in figuring out what is the general topic of a bag of words.

# Data Pre-processing

Our goal in the data pre-processing stage is to convert sentences into words, convert words to their root and removing words that are too common or too irrelevant to the purpose of our topic modeling project. We used the following techniques to reach our goal, I’m going to share the code and walk you through each stage:



1. Tokenization: Split the text into sentences and the sentences into words. Lowercase the words and remove punctuation.
2. Words that have fewer than 3 characters are 
removed.
3. All stopwords are removed. 
4. Words are lemmatized and stemmed — words in the third person are changed to first person and verbs in past and future tenses are changed into the present and are reduced to their root form.




# Tokenizing
Tokenization is always the first step before we can do any text data processing. What this means is that spaCy will segment sentences into words, punctuations, symbols and others by applying specific rules to each language. Spacy is a pre-trained natural language processing model capable of figuring out the relationship between words. You can learn more about Spacy here.

In [6]:
# Load spacy
# Make sure to restart the runtime after running installations and libraries tab
import en_core_web_sm
nlp = en_core_web_sm.load()
#nlp = spacy.load('en_core_web_lg')

/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning:

SelectableGroups dict interface is deprecated. Use select.

/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning:

SelectableGroups dict interface is deprecated. Use select.

/usr/local/lib/python3.7/dist-packages/catalogue.py:126: DeprecationWarning:

SelectableGroups dict interface is deprecated. Use select.

/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning:

SelectableGroups dict interface is deprecated. Use select.



In [7]:
# Tokenizer
tokenizer = Tokenizer(nlp.vocab)


# Custom stopwords
custom_stopwords = ['hi','\n','\n\n', '&amp;', ' ', '.', '-', 'got', "it's", 'it’s', "i'm", 'i’m', 'im', 'want', 'like', '$', '@']

# Customize stop words by adding to the default list
STOP_WORDS = nlp.Defaults.stop_words.union(custom_stopwords)

# ALL_STOP_WORDS = spacy + gensim + wordcloud
ALL_STOP_WORDS = STOP_WORDS.union(SW).union(stopwords)


tokens = []

for doc in tokenizer.pipe(df3['url_free_tweets'], batch_size=500):
    doc_tokens = []    
    for token in doc: 
        if token.text.lower() not in STOP_WORDS:
            doc_tokens.append(token.text.lower())   
    tokens.append(doc_tokens)

# Makes tokens column
df3['tokens'] = tokens

Η έξοδος ροής περικόπηκε στις τελευταίες 5000 γραμμές.
/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning:

SelectableGroups dict interface is deprecated. Use select.

/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning:

SelectableGroups dict interface is deprecated. Use select.

/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning:

SelectableGroups dict interface is deprecated. Use select.

/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning:

SelectableGroups dict interface is deprecated. Use select.

/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning:

SelectableGroups dict interface is deprecated. Use select.

/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning:

SelectableGroups dict interface is deprecated. Use select.

/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning:

SelectableGroups dict interface is depr

# Lemmatization

Lemmatization is a process where we convert words to its root word. For example: ‘Studying’ becomes ‘Study’, ‘Meeting becomes ‘Meet’, ‘Better’ and ‘Best’ become ‘Good’.The advantage of this is, we get to reduce the total number of unique words in the dictionary. As a result, the number of columns in the document-word matrix will be denser with lesser columns. The ultimate goal of lemmatization is to help the LDA model to produce better topics in the end.

In [8]:
# Make tokens a string again
df3['tokens_back_to_text'] = [' '.join(map(str, l)) for l in df3['tokens']]

def get_lemmas(text):
    '''Used to lemmatize the processed tweets'''
    lemmas = []
    
    doc = nlp(text)
    
    # Something goes here :P
    for token in doc: 
        if ((token.is_stop == False) and (token.is_punct == False)) and (token.pos_ != 'PRON'):
            lemmas.append(token.lemma_)
    
    return lemmas

df3['lemmas'] = df3['tokens_back_to_text'].apply(get_lemmas)

Η έξοδος ροής περικόπηκε στις τελευταίες 5000 γραμμές.
/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning:

SelectableGroups dict interface is deprecated. Use select.

/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning:

SelectableGroups dict interface is deprecated. Use select.

/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning:

SelectableGroups dict interface is deprecated. Use select.

/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning:

SelectableGroups dict interface is deprecated. Use select.

/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning:

SelectableGroups dict interface is deprecated. Use select.

/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning:

SelectableGroups dict interface is deprecated. Use select.

/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning:

SelectableGroups dict interface is depr

In [9]:
# Make lemmas a string again
df3['lemmas_back_to_text'] = [' '.join(map(str, l)) for l in df3['lemmas']]

# Tokenizer function
def tokenize(text):
    """
    Parses a string into a list of semantic units (words)
    Args:
        text (str): The string that the function will tokenize.
    Returns:
        list: tokens parsed out
    """
    # Removing url's
    pattern = r"http\S+"
    
    tokens = re.sub(pattern, "", text) # https://www.youtube.com/watch?v=O2onA4r5UaY
    tokens = re.sub('[^a-zA-Z 0-9]', '', text)
    tokens = re.sub('[%s]' % re.escape(string.punctuation), '', text) # Remove punctuation
    tokens = re.sub('\w*\d\w*', '', text) # Remove words containing numbers
    tokens = re.sub('@*!*\$*', '', text) # Remove @ ! $
    tokens = tokens.strip(',') # TESTING THIS LINE
    tokens = tokens.strip('?') # TESTING THIS LINE
    tokens = tokens.strip('!') # TESTING THIS LINE
    tokens = tokens.strip("'") # TESTING THIS LINE
    tokens = tokens.strip(".") # TESTING THIS LINE

    tokens = tokens.lower().split() # Make text lowercase and split it
    
    return tokens

# Apply tokenizer
df3['lemma_tokens'] = df3['lemmas_back_to_text'].apply(tokenize)

<input>:19: DeprecationWarning:

invalid escape sequence \w

<input>:19: DeprecationWarning:

invalid escape sequence \w

<input>:20: DeprecationWarning:

invalid escape sequence \$

<input>:19: DeprecationWarning:

invalid escape sequence \w

<input>:20: DeprecationWarning:

invalid escape sequence \$

<input>:19: DeprecationWarning:

invalid escape sequence \w

<input>:20: DeprecationWarning:

invalid escape sequence \$

<input>:19: DeprecationWarning:

invalid escape sequence \w

<input>:20: DeprecationWarning:

invalid escape sequence \$

<input>:19: DeprecationWarning:

invalid escape sequence \w

<input>:20: DeprecationWarning:

invalid escape sequence \$

<input>:19: DeprecationWarning:

invalid escape sequence \w

<input>:20: DeprecationWarning:

invalid escape sequence \$

<input>:19: DeprecationWarning:

invalid escape sequence \w

<input>:20: DeprecationWarning:

invalid escape sequence \$

<input>:19: DeprecationWarning:

invalid escape sequence \w

<input>:20: DeprecationW

In [10]:
df3['lemma_tokens'] 

0        [folk, say, daikon, paste, treat, cytokine, st...
1        [world, wrong, history, year, hopefully, big, ...
2        [coronavirus, sputnikv, astrazeneca, pfizerbio...
3        [fact, immutable, senator, ethically, sturdy, ...
4        [explain, need, vaccine, borisjohnson, matthan...
                               ...                        
69709    [✅, 🇷, 🇺, gamaleya, research, center, cooperat...
69710    [floydshivambu, sputnikv, africanexpertise, me...
69711    [hello, stubbornness, foolishness, face, secon...
69712    [lengthy, interview, wato, month, ago, represe...
69713       [ignore, laugh, fight, win, vaccine, sputnikv]
Name: lemma_tokens, Length: 69714, dtype: object

# Topic Modeling

## Base Model
We produce a base model first to be used to track our progress as we go through the hyper-parameter tuning stage. The LDA topic model algorithm requires a document word matrix and a dictionary as the main inputs. For the first few steps to be taken before running the LDA model, we created a dictionary, filtered the extremes and, create a corpus object which is the document matrix LDA model needs as the main input.

We prepared everything required to train the LDA model. In addition to the corpus and dictionary, we needed to provide the number of topics as well. We chose 5 for the base model. In the hyperparameter tuning stage we will reach to an optimum number of topics to be used.

In [11]:
# Create a id2word dictionary
id2word = Dictionary(df3['lemma_tokens'])
print(len(id2word))

53607


In [12]:
# Filtering Extremes
id2word.filter_extremes(no_below=2, no_above=.99)
print(len(id2word))

20284


In [13]:
# Creating a corpus object 
corpus = [id2word.doc2bow(d) for d in df3['lemma_tokens']]

In [14]:
# Instantiating a Base LDA model 
base_model = LdaMulticore(corpus=corpus, num_topics=5, id2word=id2word, workers=12, passes=5)

Η έξοδος ροής περικόπηκε στις τελευταίες 5000 γραμμές.
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning:

Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.

/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning:

Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.

/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning:

Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.

/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning:

Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.s

In [15]:
# Filtering for words 
words = [re.findall(r'"([^"]*)"',t[1]) for t in base_model.print_topics()]

In [16]:
# Create Topics
topics = [' '.join(t[0:10]) for t in words]


In [17]:

# Getting the topics

for id, t in enumerate(topics): 
    print(f"------ Topic {id} ------")
    print(t, end="\n\n")

------ Topic 0 ------
moderna vaccine covaxin today pfizer dose oxfordastrazeneca covid19 covidvaccine astrazeneca

------ Topic 1 ------
vaccine sputnikv covid19 dose moderna use pfizerbiontech india approve covid-19

------ Topic 2 ------
moderna vaccine sputnikv second dose feel day arm shoot sputnik

------ Topic 3 ------
vaccine covaxin covid19 moderna dose case covid ontario canada report

------ Topic 4 ------
vaccine moderna covaxin dose 💉 sinovac sputnikv covishield india sinopharm



By printing out the topics produced from the LDA model, we can have a rough guess about the topic related to each bag of words. It’s important to mention the bag of words have been sorted from the most relevant to the least relevant for each topic.

Model perplexity and topic coherence provide a convenient measure to judge how good a given topic model is. In my experience, topic coherence score, in particular, has been more helpful.


In [18]:

# Compute Perplexity
# a measure of how good the model is. lower the better
base_perplexity = base_model.log_perplexity(corpus)
print('\nPerplexity: ', base_perplexity) 

# Compute Coherence Score
coherence_model = CoherenceModel(model=base_model, texts=df3['lemma_tokens'], 
                                   dictionary=id2word, coherence='c_v')
coherence_lda_model_base = coherence_model.get_coherence()
print('\nCoherence Score: ', coherence_lda_model_base)

Η έξοδος ροής περικόπηκε στις τελευταίες 5000 γραμμές.
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning:

Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.

/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning:

Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.

/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning:

Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.

/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning:

Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.s


Perplexity:  -7.620836511261445

Coherence Score:  0.22632116263246874


Now that the LDA model is built, the next step is to examine the produced topics and the associated keywords. There is no better tool than pyLDAvis package’s interactive chart and is designed to work well with Google Colab and Jupyter notebooks.

In [23]:
df3

,id,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,retweets,favorites,is_retweet,processed_tweets,Polarity,Analysis,emoji_free_tweets,url_free_tweets,tokens,tokens_back_to_text,lemmas,lemmas_back_to_text,lemma_tokens
0,1340539111971516416,Rachel Roh,"La Crescenta-Montrose, CA",Aggregator of Asian American news; scanning di...,2009-04-08 17:52:46,405,1692,3247,False,2020-12-20 06:06:44,Same folks said daikon paste could treat a cyt...,['PfizerBioNTech'],Twitter for Android,0,0.0,False,folk said daikon paste could treat cytokine st...,0.292167,Negative,Same folks said daikon paste could treat a cyt...,Same folks said daikon paste could treat a cyt...,"[folks, said, daikon, paste, treat, cytokine, ...",folks said daikon paste treat cytokine storm #...,"[folk, say, daikon, paste, treat, cytokine, st...",folk say daikon paste treat cytokine storm pfi...,"[folk, say, daikon, paste, treat, cytokine, st..."
1,1338158543359250433,Albert Fong,"San Francisco, CA","Marketing dude, tech geek, heavy metal & '80s ...",2009-09-21 15:27:30,834,666,178,False,2020-12-13 16:27:13,While the world has been on the wrong side of ...,NaN,Twitter Web App,1,1.0,False,world wrong side history year hopefully bigges...,0.753310,Positive,While the world has been on the wrong side of ...,While the world has been on the wrong side of ...,"[world, wrong, history, year,, hopefully,, big...","world wrong history year, hopefully, biggest v...","[world, wrong, history, year, hopefully, big, ...",world wrong history year hopefully big vaccina...,"[world, wrong, history, year, hopefully, big, ..."
2,1337858199140118533,eli🇱🇹🇪🇺👌,Your Bed,"heil, hydra 🖐☺",2020-06-25 23:30:28,10,88,155,False,2020-12-12 20:33:45,#coronavirus #SputnikV #AstraZeneca #PfizerBio...,"['coronavirus', 'SputnikV', 'AstraZeneca', 'Pf...",Twitter for Android,0,0.0,False,coronavirus sputnikv astrazeneca pfizerbiontec...,0.364838,Negative,#coronavirus #SputnikV #AstraZeneca #PfizerBio...,#coronavirus #SputnikV #AstraZeneca #PfizerBio...,"[#coronavirus, #sputnikv, #astrazeneca, #pfize...",#coronavirus #sputnikv #astrazeneca #pfizerbio...,"[coronavirus, sputnikv, astrazeneca, pfizerbio...",coronavirus sputnikv astrazeneca pfizerbiontec...,"[coronavirus, sputnikv, astrazeneca, pfizerbio..."
3,1337855739918835717,Charles Adler,"Vancouver, BC - Canada","Hosting ""CharlesAdlerTonight"" Global News Radi...",2008-09-10 11:28:53,49165,3933,21853,True,2020-12-12 20:23:59,"Facts are immutable, Senator, even when you're...",NaN,Twitter Web App,446,2129.0,False,fact immutable senator even youre not ethicall...,0.187278,Negative,"Facts are immutable, Senator, even when you're...","Facts are immutable, Senator, even when you're...","[facts, immutable,, senator,, you're, ethicall...","facts immutable, senator, you're ethically stu...","[fact, immutable, senator, ethically, sturdy, ...",fact immutable senator ethically sturdy acknow...,"[fact, immutable, senator, ethically, sturdy, ..."
4,1337854064604966912,Citizen News Channel,NaN,Citizen News Channel bringing you an alternati...,2020-04-23 17:58:42,152,580,1473,False,2020-12-12 20:17:19,Explain to me again why we need a vaccine @Bor...,"['whereareallthesickpeople', 'PfizerBioNTech']",Twitter for iPhone,0,0.0,False,explain need vaccine whereareallthesickpeople ...,0.782983,Positive,Explain to me again why we need a vaccine @Bor...,Explain to me again why we need a vaccine @Bor...,"[explain, need, vaccine, @borisjohnson, @matth...",explain need vaccine @borisjohnson @matthancoc...,"[explain, need, vaccine, @borisjohnson, @matth...",explain need vaccine @borisjohnson @matthancoc...,"[explain, need, vaccine, borisjohnson, matthan..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69709,1382248484259123205,Russian Mission in Geneva,Geneve,Постпредство России при Отделении ООН и др. ме...,2011-07-26 08:40:47,9862,589,3

In [ ]:
#df3.to_csv("topic_data.csv", sep="\t")

In [ ]:
#Creating Topic Distance Visualization 

pyLDAvis.enable_notebook()
gensim_models.prepare(base_model, corpus, id2word)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.147733  0.080916       1        1  26.674670
0     -0.104904 -0.033985       2        1  20.987324
1     -0.010015  0.050395       3        1  19.435111
2     -0.097561  0.053681       4        1  16.968586
3      0.064746 -0.151007       5        1  15.934309, topic_info=                   Term          Freq         Total Category  logprob  loglift
24             sputnikv   6919.000000   6919.000000  Default  30.0000  30.0000
22              moderna  18380.000000  18380.000000  Default  29.0000  29.0000
3496            covaxin   9810.000000   9810.000000  Default  28.0000  28.0000
247                dose   9711.000000   9711.000000  Default  27.0000  27.0000
44               canada   1646.000000   1646.000000  Default  26.0000  26.0000
...                 ...           ...           ...      ...      ...      ...
3471  oxfordastrazeneca    548.635758   2330.709526   Topic5  -5.2391   0.3902
38                covid    588.938928   3994.667689   Topic5  -5.1682  -0.0777
429               today    558.172298   4306.087578   Topic5  -5.2219  -0.2064
24             sputnikv    610.886190   6919.452657   Topic5  -5.1316  -0.5905
181            covid-19    508.306871   3841.689807   Topic5  -5.3155  -0.1859

[417 rows x 6 columns], token_table=       Topic      Freq  Term
term                        
16418      3  0.956776  0.03
26         1  0.490327     1
26         2  0.040658     1
26         3  0.066752     1
26         4  0.174163     1
...      ...       ...   ...
1861       1  0.108111     🤣
1861       2  0.019305     🤣
1861       3  0.826275     🤣
1861       4  0.007722     🤣
1861       5  0.046333     🤣

[1199 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 1, 2, 3, 4])

So how to infer pyLDAvis’s output?
Each bubble on the left-hand side plot represents a topic. The larger the bubble, the more prevalent is that topic. A good topic model will have fairly big, non-overlapping bubbles scattered throughout the chart instead of being clustered in one quadrant. A model with too many topics, will typically have many overlaps, small sized bubbles clustered in one region of the chart.
Alright, if we move the cursor over one of the bubbles, the words and bars on the right-hand side will update. These words are the salient keywords that form the selected topic. We have successfully built a good looking topic model. Up next, we will improve upon this model (Hyperparameter Tuning) by using Scikit-learn’s Grid Search and then we will focus on how to arrive at the optimal number of topics and other variables in within the LDA model.

# Hyperparameter Tuning

## Grid Search
The most important tuning parameter for LDA models is n_components (number of topics). In addition, we are going to search learning_decay (which controls the learning rate) as well. Besides these, other possible search params could be learning_offset (down weight early iterations. Should be > 1) and max_iter. These could be worth experimenting if you have enough time and computing resources.
Be warned, the grid search constructs multiple LDA models for all possible combinations of param values in the param_grid dict. So, this process can consume a lot of time and resources.

In [20]:
vectorizer = CountVectorizer()
data_vectorized = vectorizer.fit_transform(df3['lemmas_back_to_text'])

In [ ]:
# Define Search Param
search_params = {'n_components': [10, 15, 20, 25, 30], 'learning_decay': [.5, .7, .9]}

# Init the Model
lda = LatentDirichletAllocation()

# Init Grid Search Class
model = GridSearchCV(lda, param_grid=search_params)

# Do the Grid Search
model.fit(data_vectorized)
GridSearchCV(cv=None, error_score='raise',
             estimator=LatentDirichletAllocation(batch_size=128, 
                                                 doc_topic_prior=None,
                                                 evaluate_every=-1, 
                                                 learning_decay=0.7, 
                                                 learning_method=None,
                                                 learning_offset=10.0, 
                                                 max_doc_update_iter=100, 
                                                 max_iter=10,
                                                 mean_change_tol=0.001, 
                                                 n_components=10, 
                                                 n_jobs=1,
                                                 perp_tol=0.1, 
                                                 random_state=None,
                                                 topic_word_prior=None, 
                                                 total_samples=1000000.0, 
                                                 verbose=0),
             iid=True, n_jobs=1,
             param_grid={'n_topics': [10, 15, 20, 30], 
                         'learning_decay': [0.5, 0.7, 0.9]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
             scoring=None, verbose=0)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:440: DeprecationWarning:

`np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:93: DeprecationWarning:

`np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes

In [ ]:
# Best Model
best_lda_model = model.best_estimator_

# Model Parameters
print("Best Model's Params: ", model.best_params_)

# Log Likelihood Score
print("Best Log Likelihood Score: ", model.best_score_)

# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(data_vectorized))

We know there is a high chance number of topics is much more than 10 but through the grid search we found out 10 topics performs better than other number of topics. This makes us think deeply about the results, and we figured Scikit-learn’s grid search tracks perplexity not coherence value and for our use case, coherence value delivers the best result. In the next step we’re going to get to an optimal number of topics.

## Optimum number of topics

Our approach to finding the optimal number of topics was to build many LDA models with different values of number of topics and pick the one that gives the highest coherence value.


In [26]:
#Defining a function to loop over number of topics to be used to find an 
#optimal number of tipics
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the 
    LDA model with respective number of topics
    """
    coherence_values_topic = []
    model_list_topic = []
    for num_topics in range(start, limit, step):
        model = LdaMulticore(corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list_topic.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values_topic.append(coherencemodel.get_coherence())

    return model_list_topic, coherence_values_topic

In [28]:
# Can take a long time to run.
model_list_topic, coherence_values_topic = compute_coherence_values(dictionary=id2word,
                                                        corpus=corpus,
                                                        texts=df3['lemma_tokens'],
                                                        start=2, limit=200, step=6)

Η έξοδος ροής περικόπηκε στις τελευταίες 5000 γραμμές.
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning:

Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.

/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning:

Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.

/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning:

Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.

/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning:

Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.s

Choosing a number of topic that marks the end of a rapid growth of topic coherence usually offers meaningful and interpretable topics. Picking an even higher value can sometimes provide more granular sub-topics.If you see the same keywords being repeated in multiple topics, it’s probably a sign that the ‘number of topics’ is too large. By following these principles we chose 68 topics as an optimal number of topics for our use case.

In [32]:
model_list_topic

## Optimal Number of Passes

Passes, chunksize and update_every are parameters that have a EM/Variational relationship together. We are not going to go into the details of EM/Variational Bayes here, but if you are curious, check out this google forum post and the paper it references here. For our use case, hyperparameter tuning number of passes would suffice considering chuncksize is not substantial and update_every would not change much in the final results.


In [29]:
model_5_2 = LdaMulticore(corpus=corpus,
                       id2word=id2word,
                       num_topics=68,
                       random_state=42,
                       chunksize=2000,
                       passes=25,
                       decay=0.5,
                       iterations=70)

Η έξοδος ροής περικόπηκε στις τελευταίες 5000 γραμμές.
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning:

Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.

/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning:

Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.

/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning:

Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.

/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning:

Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.s

In [50]:
words = [re.findall(r'"([^"]*)"',t[1]) for t in model_5_2.print_topics()]
topics = [' '.join(t[0:10]) for t in words]
for id, t in enumerate(topics): 
    print(f"------ Topic {id} ------")
    print(t, end="\n\n")


------ Topic 0 ------
death way follow medical possible court city 🤔 small stfbrasil

------ Topic 1 ------
take dose support delivery astrazenaca confirm covid-19 situation point record

------ Topic 2 ------
india vaccine crore stock run analysis get release supreme covid

------ Topic 3 ------
open russia usual lockdown justintrudeau high well risk long celliottability

------ Topic 4 ------
april sars cov-2 2021 eligible ✔ deal tweet sign 2020

------ Topic 5 ------
know try w 🚀 7 article create amp note partnership

------ Topic 6 ------
bharatbiotech icmrdelhi break seruminstindia order 3rd tuesday general permission lobby

------ Topic 7 ------
russian sputnikv multiple vaccine expert regulator export clear panel european

------ Topic 8 ------
minister modi institute trust prime central understand vaccine say gang

------ Topic 9 ------
wait let moderna save later round place send win list

------ Topic 10 ------
trudeau book low nation demand national love bjp authority 11

--

In [51]:
# Compute Perplexity
# a measure of how good the model is. lower the better
model_5_2_perplexity = model_5_2.log_perplexity(corpus)
print('\nPerplexity: ', model_5_2_perplexity) 



Η έξοδος ροής περικόπηκε στις τελευταίες 5000 γραμμές.
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning:

Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.

/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning:

Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.

/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning:

Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.

/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning:

Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.s


Perplexity:  -8.197159627702833


In [52]:
# Compute Coherence Score
coherence_model = CoherenceModel(model=model_5_2, texts=df3['lemma_tokens'], 
                                   dictionary=id2word, coherence='c_v')
coherence_lda_model_5_2 = coherence_model.get_coherence()
print('\nCoherence Score: ', coherence_lda_model_5_2)
pyLDAvis.enable_notebook()
gensim_models.prepare(model_5_2, corpus, id2word)


Coherence Score:  0.47558663114067


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
64    -0.183552  0.134252       1        1  3.874975
54    -0.172365 -0.158964       2        1  3.812996
14    -0.249918 -0.139411       3        1  3.808117
13    -0.258282 -0.062444       4        1  3.361547
45    -0.161049  0.026048       5        1  3.259600
...         ...       ...     ...      ...       ...
52     0.125924  0.035208      64        1  0.910137
55     0.065694 -0.060101      65        1  0.898745
50     0.109577  0.015880      66        1  0.890846
56    -0.004389 -0.021732      67        1  0.815866
63     0.107715 -0.001604      68        1  0.669914

[68 rows x 5 columns], topic_info=            Term          Freq         Total Category  logprob  loglift
3496     covaxin   9105.000000   9105.000000  Default  30.0000  30.0000
24      sputnikv   8487.000000   8487.000000  Default  29.0000  29.0000
22       moderna  18809.000000  18809.000000  Default  28.0000  28.0000
655        india   4192.000000   4192.000000  Default  27.0000  27.0000
247         dose   9323.000000   9323.000000  Default  26.0000  26.0000
...          ...           ...           ...      ...      ...      ...
4398       pride     29.380977     30.359581  Topic68  -4.9971   4.9730
999   superpower     28.316710     29.295313  Topic68  -5.0340   4.9718
2586       dubai     28.246046     29.224650  Topic68  -5.0365   4.9717
1304  assessment     27.690352     28.668956  Topic68  -5.0564   4.9710
962         time     48.205759   1128.849548  Topic68  -4.5020   1.8523

[2462 rows x 6 columns], token_table=       Topic      Freq      Term
term                            
6937      35  0.959655  #moderna
1274      52  0.998742         +
6880      35  0.983382         0
16418     20  0.991675      0.03
6076      57  0.995019      0.04
...      ...       ...       ...
1998      52  0.979719         🤷
900       46  0.986715         🥰
477       62  0.996465         🥳
1116      12  0.991263         🦠
4721      56  0.987110         🧐

[2696 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[65, 55, 15, 14, 46, 38, 68, 20, 27, 23, 54, 19, 29, 2, 28, 62, 26, 1, 49, 6, 39, 24, 21, 9, 31, 8, 60, 4, 30, 10, 58, 12, 25, 35, 42, 7, 50, 37, 66, 33, 43, 16, 34, 47, 52, 45, 11, 59, 32, 48, 41, 22, 67, 3, 44, 5, 17, 18, 61, 63, 40, 36, 13, 53, 56, 51, 57, 64])